In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
SCOPETYPE = 'OPENvADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'AVRCRYPTOLIB'

In [ ]:
def load_bitstream():
    %run "../Setup_Scripts/Setup_Generic.ipynb"
    scope.scopetype.cwFirmwareConfig[0xACE2].loader.setFPGAMode("debug")
    print("Mode: " + str(scope.scopetype.cwFirmwareConfig[0xACE2].loader._release_mode))
    print("Mode: " + str(scope.scopetype.cwFirmwareConfig[0xACE2].loader.fpga_bitstream()))
    input("powercycle done?")
    %run "../Setup_Scripts/Setup_Generic.ipynb"
    
load_bitstream()

In [ ]:
import holoviews as hv
import pandas as pd
import numpy as np

from holoviews.streams import Pipe, Buffer
from scipy.ndimage.filters import uniform_filter1d
from tqdm.notebook import trange

import chipwhisperer.analyzer as cwa
import chipwhisperer as cw
from chipwhisperer.common.traces import Trace

class real_time_plot:
    def __init__(self, plot_len, title="Plot"):
        hv.extension('bokeh')
        st = pd.DataFrame({'y':[]}, columns=['y'])
        self.dfstream = Buffer(st, length=plot_len, index=False)
        curve_dmap = hv.DynamicMap(hv.Curve, streams=[self.dfstream])
        curve_dmap.opts(tools=["hover"], width=800, height=600, title=title)
        display(curve_dmap)
        self.curve = curve_dmap
    def send(self, data):
        if hasattr(data, 'wave'):
            d = data.wave
        else:
            d = data
        self.dfstream.send(pd.DataFrame({'y':d}))

def powercycle_target(scope):
    scope.io.tio1 = "gpio_low"
    time.sleep(0.0001)
    scope.io.tio1 = "gpio_high"
    
def print_ec_conf():
    print("==== Trigger ====")
    print(f"cw reports running:\t\t {scope.EC.check_status()}")
    print(f"cw reports edge_num:\t\t {scope.EC.edge_num}")
    print(f"cw reports settling_time:\t {scope.EC.settling_time}")
    print(f"cw reports pretrigger_ctr:\t {scope.EC.pretrigger_ctr}")
    print(f"cw reports edge_type:\t\t {scope.EC.edge_type}")
    print(f"cw reports threshold:\t\t {scope.EC.threshold}")
    print(f"cw reports trigger module:\t {scope.trigger.module}")
    
def setup_cw():    
    scope.adc.offset = offset
    scope.gain.gain = 50
    scope.adc.samples = nr_samples
    # scope.adc.presamples = 2000
    
    # per docs has to be like that for DecodeIO and SAD
    scope.adc.basic_mode = "rising_edge"
    
def setup_glitcher():
    scope.glitch.width = 0.2
    scope.glitch.width_fine = 200
    scope.glitch.trigger_src = "ext_single"
    scope.glitch.repeat = 1
    scope.glitch.output = "glitch_only"

def setup_ec():
    setup_cw()
    scope.EC.threshold = 0.1
    scope.EC.settling_time = 127
    scope.EC.pretrigger_ctr = 0
    scope.EC.edge_num = 0
    scope.EC.edge_type = "rising_edge"
    scope.trigger.module = "EC"
    

    
def run_test_cycle():
    plot = real_time_plot(plot_len=20000)

    for i in trange(5, desc='Capturing traces'):    
        powercycle_target(scope)
        scope.arm()
        scope.capture()
        trace = scope.get_last_trace()
        if trace is None:
            continue
        plot.dfstream.clear()
        plot.send(trace)

nr_samples = 24400  
offset = 0
traces = 20

setup_ec()
print_ec_conf()

# Run EC trigger
run_test_cycle()

In [ ]:
def setup_basic():
    setup_cw()

    scope.adc.basic_mode = "low"
    scope.trigger.module = "basic"
    scope.trigger.triggers = "tio4"

def collect_reference_trace(traces, nr_samples, offset):
    total_len = traces*nr_samples
    plot = real_time_plot(total_len)

    new_trace = np.zeros(0)
    for i in trange(traces, desc='Capturing traces'):
        cur_offset = int(offset + nr_samples*i)
        scope.adc.offset = cur_offset

        scope.arm()
        powercycle_target(scope)
        scope.capture()

        trace = scope.get_last_trace()
        if trace is None:
            continue
        new_trace = np.append(new_trace, trace)
    plot.send(new_trace)
    return new_trace

setup_basic()
# Show a longer trace to get a sense of what's going on in the target
collect_reference_trace(traces, nr_samples, offset)